In [63]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

import re
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import json

import keras as k
from keras import layers
from keras.layers import recurrent
from keras.layers.embeddings import Embedding
from keras.models import Model

import tensorflow as tf

from sklearn.metrics import hamming_loss, precision_score,\
                             recall_score, accuracy_score
from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial.distance import cosine
from tensorflow.keras.metrics import CosineSimilarity

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def tokenize(sent):
  return [x.strip() for x in re.split(r'(\W+)', sent) if x.strip()]

In [0]:
with open('drive/My Drive/final_project/train-v2.0.json', 'r') as f:
  content = json.loads(f.read())
type(content)

dict

In [0]:
print(content.keys())

dict_keys(['version', 'data'])


In [0]:
data = content['data']
data[0].keys()

dict_keys(['title', 'paragraphs'])

In [0]:
data[320]['paragraphs'][0]

{'context': 'The modern English word green comes from the Middle English and Anglo-Saxon word grene, from the same Germanic root as the words "grass" and "grow". It is the color of living grass and leaves and as a result is the color most associated with springtime, growth and nature. By far the largest contributor to green in nature is chlorophyll, the chemical by which plants photosynthesize and convert sunlight into chemical energy. Many creatures have adapted to their green environments by taking on a green hue themselves as camouflage. Several minerals have a green color, including the emerald, which is colored green by its chromium content.',
 'qas': [{'answers': [{'answer_start': 326, 'text': 'chlorophyll'}],
   'id': '5729604faf94a219006aa341',
   'is_impossible': False,
   'question': 'What, in nature, is most likely to make things green?'},
  {'answers': [{'answer_start': 522, 'text': 'camouflage'}],
   'id': '5729604faf94a219006aa342',
   'is_impossible': False,
   'question

In [0]:
def parse_data(data):
  vocab_set = set()
  vocab = {}
  triplex_list = []
  context_list = []
  question_list = []
  answer_list = []

  # Context and questions extracting
  for topic in data:
    for part in topic['paragraphs']:
      blocks = part['qas']
      for block in blocks:
        context = part['context']
        vocab_set |= set(tokenize(context))
        context_list.append(context)
        vocab_set |= set(tokenize(block['question']))
        question_list.append(block['question'])

  # Making dictionary with shape {'token': number}, where numbers are in range 1..
  i = 1
  for token in vocab_set:
    vocab[token] = i
    i += 1
  
  # Context vectorization and finding of context_maxlen
  context_vectors = []
  context_maxlen = 0
  for context in context_list:
    vectorized_context = []
    tokens = tokenize(context)
    for token in tokens:
      vectorized_context.append(vocab[token])
    context_vectors.append(vectorized_context)
    if len(tokens) > context_maxlen:
      context_maxlen = len(tokens)
  context_vectors = pad_sequences(context_vectors, maxlen=context_maxlen, padding='post')

  # Answers extracting and vectorization
  for topic in data:
    for part in topic['paragraphs']:
      blocks = part['qas']
      for block in blocks:
        answer = np.zeros(context_maxlen + 1)
        if len(block['answers']) == 1:
          answer_start = block['answers'][0]['answer_start']
          text = block['answers'][0]['text']
          answer[answer_start:answer_start + len(text)] = 1
        answer_list.append(answer)
    answer_vectors = pad_sequences(answer_list, maxlen=context_maxlen, padding='post')
  
  # Question vectorization and question_maxlen finding 
  question_vectors = []
  question_maxlen = 0
  for question in question_list:
    vectorized_question = []
    tokens = tokenize(question)
    for token in tokens:
      vectorized_question.append(vocab[token])
    question_vectors.append(vectorized_question)
    if len(tokens) > question_maxlen:
      question_maxlen = len(tokens)
  question_vectors = pad_sequences(question_vectors, maxlen=question_maxlen, padding='post')


  return context_vectors, question_vectors, answer_vectors, vocab, context_maxlen, question_maxlen

In [0]:
context_vectors, question_vectors, answer_vectors, vocab, context_maxlen, question_maxlen = parse_data(data)

In [0]:
with open('drive/My Drive/final_project/context_vectors.pickle', 'wb') as f:
  pickle.dump(context_vectors, f)
with open('drive/My Drive/final_project/question_vectors.pickle', 'wb') as f:
  pickle.dump(question_vectors, f)
with open('drive/My Drive/final_project/answer_vectors.pickle', 'wb') as f:
  pickle.dump(answer_vectors, f)
other = {'vocab': vocab, 'context_maxlen': context_maxlen, 'question_maxlen': question_maxlen}
with open('drive/My Drive/final_project/other.pickle', 'wb') as f:
  pickle.dump(other, f)


In [0]:
with open('drive/My Drive/final_project/context_vectors.pickle', 'rb') as f:
  context_vectors = pickle.load(f)
with open('drive/My Drive/final_project/question_vectors.pickle', 'rb') as f:
  question_vectors = pickle.load(f)
with open('drive/My Drive/final_project/answer_vectors.pickle', 'rb') as f:
  answer_vectors = pickle.load(f)
with open('drive/My Drive/final_project/other.pickle', 'rb') as f:
  other = pickle.load(f)
vocab = other['vocab']
context_maxlen = other['context_maxlen']
question_maxlen = other['question_maxlen']


In [0]:
print('context ', type(context_vectors))
print('question ', type(question_vectors))
print('answer ', type(answer_vectors))
print('context ', context_vectors.shape)
print('question ', question_vectors.shape)
print('answer ', answer_vectors.shape)
print('context_maxlen= ', context_maxlen)
print('question_maxlen= ', question_maxlen)
print('Length of vocabulary= ', len(vocab))


context  <class 'numpy.ndarray'>
question  <class 'numpy.ndarray'>
answer  <class 'numpy.ndarray'>
context  (130319, 844)
question  (130319, 60)
answer  (130319, 844)
context_maxlen=  844
question_maxlen=  60
Length of vocabulary=  99372


In [0]:
from sklearn.model_selection import train_test_split
context_train, context_test, question_train, question_test,\
  answer_train, answer_test = train_test_split(context_vectors,
  question_vectors, answer_vectors, test_size=0.2, random_state=42)

print('context_train', type(context_train), context_train.shape)
print('context_test', type(context_test), context_test.shape)
print('question_train', type(question_train), question_train.shape)
print('question_test', type(question_test), question_test.shape)
print('answer_train', type(answer_train), answer_train.shape)
print('answer_test', type(answer_test), answer_test.shape)


context_train <class 'numpy.ndarray'> (104255, 844)
context_test <class 'numpy.ndarray'> (26064, 844)
question_train <class 'numpy.ndarray'> (104255, 60)
question_test <class 'numpy.ndarray'> (26064, 60)
answer_train <class 'numpy.ndarray'> (104255, 844)
answer_test <class 'numpy.ndarray'> (26064, 844)


In [94]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 100
CONTEXT_HIDDEN_SIZE = 200
QUESTION_HIDDEN_SIZE = 200
BATCH_SIZE = 256
EPOCHS = 10
vocab_size = len(vocab) + 1
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           CONTEXT_HIDDEN_SIZE,
                                                           QUESTION_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 100, 200, 200


In [95]:
print('Build model...')

context = layers.Input(shape=(context_maxlen,), dtype='int32')
encoded_context = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(context)
encoded_context = RNN(CONTEXT_HIDDEN_SIZE)(encoded_context)

question = layers.Input(shape=(question_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = RNN(QUESTION_HIDDEN_SIZE)(encoded_question)

merged = layers.concatenate([encoded_context, encoded_question])
preds = layers.Dense(context_maxlen, activation='sigmoid')(merged)

model = Model([context, question], preds)
model.compile(optimizer='rmsprop',
              loss=tf.nn.sigmoid_cross_entropy_with_logits,
              metrics=['accuracy', \
                       CosineSimilarity(axis=1)])

model.summary()

Build model...
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 844)          0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_31 (Embedding)        (None, 844, 100)     9937300     input_31[0][0]                   
__________________________________________________________________________________________________
embedding_32 (Embedding)        (None, 60, 100)      9937300     input_32[0][0]                   
____________________________________________________________________________

In [0]:
from keras.callbacks import ModelCheckpoint

callback = ModelCheckpoint(filepath='/content/drive/My Drive/final_project/weights_file_30',
              monitor='val_loss',
              mode='auto',
              save_best_only=True)

print('Training')
history = model.fit([context_train, question_train], answer_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05,
          callbacks=[callback])


In [0]:
preds_1 = model.predict([context_train, question_train])

In [88]:
print(preds_1.shape)
print(answer_train.shape)
print(preds_1[10])
print(answer_train[10])

(104255, 844)
(104255, 844)
[1.18392343e-07 1.18550290e-07 1.19494800e-07 1.19176583e-07
 1.20197470e-07 1.20464733e-07 1.18213414e-07 1.19060374e-07
 1.18872045e-07 1.19386243e-07 1.19240141e-07 1.18184325e-07
 1.19378271e-07 1.19167382e-07 1.19947956e-07 1.19380317e-07
 1.19562955e-07 1.17541163e-07 1.17795565e-07 1.19408327e-07
 1.18637033e-07 1.17165904e-07 1.17860388e-07 1.18238319e-07
 1.18626396e-07 1.20242078e-07 1.16504602e-07 1.18031132e-07
 1.19302925e-07 1.17691798e-07 1.18888821e-07 1.20021070e-07
 1.20056569e-07 1.19649187e-07 1.20366892e-07 1.19551558e-07
 1.20068009e-07 1.17207478e-07 1.20141550e-07 1.18308250e-07
 1.18780818e-07 1.20346002e-07 1.20629707e-07 1.18761669e-07
 1.18604575e-07 1.19773759e-07 1.20637196e-07 1.20458537e-07
 1.20187153e-07 1.19614967e-07 1.18783987e-07 1.19908265e-07
 1.20963563e-07 1.20021767e-07 1.19376907e-07 1.18451510e-07
 1.19282902e-07 1.18219042e-07 1.20329020e-07 1.20272460e-07
 1.19372913e-07 1.21499440e-07 1.18024836e-07 1.20236791e

In [71]:
accuracy=np.mean(history.history['accuracy'])
val_accuracy=np.mean(history.history['val_accuracy'])
loss=np.mean(history.history['loss'])
val_loss=np.mean(history.history['val_loss'])
cosine_similarity=np.mean(history.history['cosine_similarity'])
val_cosine_similarity=np.mean(history.history['val_cosine_similarity'])
print('val_accuracy= ', val_accuracy)
print('val_loss= ', val_loss)
print('val_cosine_similarity= ', val_cosine_similarity)

val_accuracy=  0.000709764042403549
val_loss=  0.6931475123951171
val_cosine_similarity=  0.08889962434768676


In [97]:
h = history.history
h

{'accuracy': [0.000959189,
  0.0013428646,
  0.0013529614,
  0.0013529614,
  0.0013933483,
  0.0013933483,
  0.0013933483,
  0.0013933483,
  0.0013933483,
  0.0013933483],
 'cosine_similarity': [0.08596261,
  0.0883613,
  0.088790566,
  0.08891382,
  0.088985905,
  0.08905889,
  0.08910714,
  0.089145,
  0.08916608,
  0.08916073],
 'loss': [0.6957988121626978,
  0.693147599697113,
  0.6931474804878235,
  0.6931474802158045,
  0.6931474802158045,
  0.6931474802158045,
  0.6931474802158045,
  0.6931474802158045,
  0.6931474802158045,
  0.6931474802158045],
 'val_accuracy': [0.0013427968369796872,
  0.0005754843587055802,
  0.0011509687174111605,
  0.0005754843587055802,
  0.0005754843587055802,
  0.0005754843587055802,
  0.0005754843587055802,
  0.0005754843587055802,
  0.0005754843587055802,
  0.0005754843587055802],
 'val_cosine_similarity': [0.08785398304462433,
  0.08859404176473618,
  0.08884090185165405,
  0.08896390348672867,
  0.08903752267360687,
  0.08908645063638687,
  0.08912

In [73]:
print('Evaluation')
test_loss, test_accuracy, test_cosine_similarity = model.evaluate([context_test, question_test], answer_test,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy / test_cosine_similarity = {:.4f} / {:.4f} / {:.4f}'.format(loss, accuracy, cosine_similarity))

Evaluation
26064/26064 [==============================] - 13s 508us/step
Test loss / test accuracy / test_cosine_similarity = 0.6934 / 0.0013 / 0.0887


In [0]:
from tensorflow.keras.models import model_from_json

json_file = '/content/drive/My Drive/final_project/model.json'
model_json_1 = model.to_json()

with open(json_file, 'w') as f:
  f.write(model_json_1)
